eval time - run 100 times on 100kb mol

loc time - deepom vs falcon

aligner time - our vs bionano

In [77]:
from deepom.bionano_utils import MoleculeSelector

selector = MoleculeSelector()
selector.top_mol_num = 10
selector.select_molecules()

KeyboardInterrupt: 

In [ ]:
from deepom.config import Config
from deepom.localizer import LocalizerModule

localizer_module = LocalizerModule()
localizer_module.checkpoint_search_dir = Config.CHECKPOINT_SEARCH_DIR
localizer_module.load_checkpoint = True
localizer_module.init_ensure_module()

BasicUNet features: (32, 32, 64, 128, 256, 32).
loading checkpoint:  /home/ynogin/Dropbox/code/DeepOM/data/pretrained/checkpoint.pickle 




In [ ]:
localizer_module_cpu = LocalizerModule()
localizer_module_cpu.device = 'cpu'
localizer_module_cpu.checkpoint_search_dir = Config.CHECKPOINT_SEARCH_DIR
localizer_module_cpu.load_checkpoint = True
localizer_module_cpu.init_ensure_module()

BasicUNet features: (32, 32, 64, 128, 256, 32).
loading checkpoint:  /home/ynogin/Dropbox/code/DeepOM/data/pretrained/checkpoint.pickle 




In [ ]:
from deepom.bionano_compare import BionanoCompare

compare = BionanoCompare()
compare.read_cmap()
compare.make_refs()
compare.refs

1     [14453.0, 19686.0, 22554.0, 31191.0, 33547.0, ...
2     [15924.0, 22064.0, 28594.0, 34027.0, 34664.0, ...
3     [12734.0, 13105.0, 17438.0, 20151.0, 26344.0, ...
4     [12985.0, 14575.0, 23028.0, 27375.0, 35689.0, ...
5     [19314.0, 19924.0, 22572.0, 48655.0, 52154.0, ...
6     [76216.0, 78491.0, 80268.0, 84078.0, 92283.0, ...
7     [10487.0, 12916.0, 15501.0, 17193.0, 18142.0, ...
8     [61805.0, 66488.0, 70256.0, 73189.0, 79269.0, ...
9     [14566.0, 19799.0, 22667.0, 30973.0, 33330.0, ...
10    [18514.0, 19074.0, 29291.0, 29794.0, 32222.0, ...
11    [60457.0, 62233.0, 66043.0, 73015.0, 81386.0, ...
12    [14568.0, 19721.0, 24617.0, 33245.0, 35595.0, ...
13    [16004725.0, 16010361.0, 16021642.0, 16136233....
14    [16004725.0, 16010361.0, 16021642.0, 16031132....
15    [17050953.0, 17052426.0, 17058646.0, 17060180....
16    [14134.0, 19375.0, 28229.0, 38092.0, 44607.0, ...
17    [66653.0, 70073.0, 84695.0, 84709.0, 90870.0, ...
18    [18868.0, 19428.0, 29621.0, 30124.0, 32552

In [ ]:
from matplotlib.pyplot import eventplot, imshow, figure, xlim
molecule = selector.selected[0]
# figure(figsize=(30, 3))
image_input = molecule.bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
# imshow(image_input, aspect="auto", cmap="gray")


Molecule info

In [ ]:
molecule.xmap_item.xmap_record[["Length"]]
print("molecule length: ", molecule.xmap_item.xmap_record[["Length"]].item() / 1000, "kb")
print("image shape: ", image_input.shape[0], "x" , image_input.shape[1], "pixels")


molecule length:  686.64 kb
image shape:  5 x 1830 pixels


In [ ]:
from deepom.falcon_compare import Falcon
falcon = Falcon()
falcon.start()

/home/ynogin/Dropbox/code/FALCON2D


In [79]:
from tqdm.auto import tqdm

In [92]:
%%timeit -n 1 -r 10
inference_item = localizer_module_cpu.inference_item(image_input)

9.3 ms ± 756 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [91]:
%%timeit -n 1 -r 10
inference_item = localizer_module.inference_item(image_input)

7.37 ms ± 1.17 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:
import imageio
import numpy as np
from skimage import img_as_uint

image_input = molecule.bionano_image.segment_image[0]
image_input = image_input - np.min(image_input)
image_input = image_input / np.max(image_input)
image_input = img_as_uint(image_input)
source_width = image_input.shape[0] // 2 + 1
target_width = 9
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]

image_file = "/tmp/runtime_test_image.tif"
imageio.mimwrite(image_file, image_input[None])

In [93]:
%%timeit -n 1 -r 10
locs = falcon(image_file)

FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
FALCON GPU start...
frames no. 1 ~ 1 are reconstructing...
5.74 s ± 180 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [94]:
self = BionanoCompare()
self.data_prep.selector.top_mol_num = 1
self.data_prep.num_crops_per_size = 100
self.data_prep.crop_size_range_bp = 100 * 1000, 100 * 1000
self.data_prep.num_sizes = 1
self.data_prep.selector.run_ids = np.arange(8) + 1
self.init_run()
self.read_cmap()
self.make_refs()

self.data_prep.make_crops()
self.data_prep.print_crops_report()
self.data_prep.make_crops_bnx()

/home/ynogin/out/bionano_compare/20230129T155925Z/20230129T155925Z
reading:  /home/ynogin/data/bionano_data/bnx.db


selected: 0it [00:00, ?it/s]

crop_sizes [100000.]


crops: 0it [00:00, ?it/s]

{'num_original_molecules': 1}


Bionano Pipeline

In [95]:
%%timeit -n 1 -r 10
self.run_bionano_on_bnx()

/home/ynogin/bionano_sw/tools/pipeline/Solve3.7_03302022_283/RefAligner/1.0/RefAligner -i /home/ynogin/out/refaligner_out/20230129T155931Z/20230129T155931Z.bnx -ref /home/ynogin/data/bionano_runs/2022-04-19/hg38_DLE1_0kb_0labels.cmap -o /home/ynogin/out/refaligner_out/20230129T155931Z/20230129T155931Z -f -BestRef 1 -usecolor 1 -A 2 -T 1 -S -1000


refaligner:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ynogin/bionano_sw/tools/pipeline/Solve3.7_03302022_283/RefAligner/1.0/RefAligner -i /home/ynogin/out/refaligner_out/20230129T160008Z/20230129T160008Z.bnx -ref /home/ynogin/data/bionano_runs/2022-04-19/hg38_DLE1_0kb_0labels.cmap -o /home/ynogin/out/refaligner_out/20230129T160008Z/20230129T160008Z -f -BestRef 1 -usecolor 1 -A 2 -T 1 -S -1000


refaligner:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ynogin/bionano_sw/tools/pipeline/Solve3.7_03302022_283/RefAligner/1.0/RefAligner -i /home/ynogin/out/refaligner_out/20230129T160043Z/20230129T160043Z.bnx -ref /home/ynogin/data/bionano_runs/2022-04-19/hg38_DLE1_0kb_0labels.cmap -o /home/ynogin/out/refaligner_out/20230129T160043Z/20230129T160043Z -f -BestRef 1 -usecolor 1 -A 2 -T 1 -S -1000


...

/home/ynogin/bionano_sw/tools/pipeline/Solve3.7_03302022_283/RefAligner/1.0/RefAligner -i /home/ynogin/out/refaligner_out/20230129T160424Z/20230129T160424Z.bnx -ref /home/ynogin/data/bionano_runs/2022-04-19/hg38_DLE1_0kb_0labels.cmap -o /home/ynogin/out/refaligner_out/20230129T160424Z/20230129T160424Z -f -BestRef 1 -usecolor 1 -A 2 -T 1 -S -1000


refaligner:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ynogin/bionano_sw/tools/pipeline/Solve3.7_03302022_283/RefAligner/1.0/RefAligner -i /home/ynogin/out/refaligner_out/20230129T160502Z/20230129T160502Z.bnx -ref /home/ynogin/data/bionano_runs/2022-04-19/hg38_DLE1_0kb_0labels.cmap -o /home/ynogin/out/refaligner_out/20230129T160502Z/20230129T160502Z -f -BestRef 1 -usecolor 1 -A 2 -T 1 -S -1000


refaligner:   0%|          | 0/1 [00:00<?, ?it/s]

36.9 s ± 864 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


DeepOM Pipeline

In [96]:
%%timeit -n 1 -r 10
self.run_aligner()

qry items:   0%|          | 0/200 [00:00<?, ?it/s]

BasicUNet features: (32, 32, 64, 128, 256, 32).
loading checkpoint:  /home/ynogin/Dropbox/code/DeepOM/data/pretrained/checkpoint.pickle 




/home/ynogin/miniconda3/envs/deepom/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


alignments:   0%|          | 0/4800 [00:00<?, ?it/s]

len(self.aligner_items)=100


...

len(self.aligner_items)=100


qry items:   0%|          | 0/200 [00:00<?, ?it/s]

/home/ynogin/miniconda3/envs/deepom/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:107: FutureWarning: <class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
  warn_deprecated(obj, msg, warning_category)


alignments:   0%|          | 0/4800 [00:00<?, ?it/s]

len(self.aligner_items)=100
35.7 s ± 1.02 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [127]:
localizer_module.module.down_2

Down(
  (max_pooling): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convs): TwoConv(
    (conv_0): Convolution(
      (conv): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (adn): ADN(
        (N): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (D): Dropout(p=0.0, inplace=False)
        (A): LeakyReLU(negative_slope=0.1, inplace=True)
      )
    )
    (conv_1): Convolution(
      (conv): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (adn): ADN(
        (N): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (D): Dropout(p=0.0, inplace=False)
        (A): LeakyReLU(negative_slope=0.1, inplace=True)
      )
    )
  )
)